In [1]:
import os
import sys
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis\read")
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis")

import numpy as np
import matplotlib.pyplot as plt
import scipy

from read_scope_data import read_trc_data
from data_analysis_utils import Photons
from plot_utils import plot_stft, plot_counts_per_bin, plot_photon_detection

%matplotlib qt

In [7]:
# Define parameters for file selection
date = "20241102"
file_number = "00012"
position = "p30-z13-x200"  # Change this for different positions

# Define base pattern for file names
xray_pattern = f"C{{channel}}--E-ring-{position}-xray--{file_number}.trc"
bdot_pattern = f"C{{channel}}--E-ring-{position}-Bdot--{file_number}.trc"

print(f"Looking for files with {position}, shot: {file_number}")

# read data from xray detector scope
base_dir = os.path.join(r"E:\x-ray", date)
for channel in ["2", "3"]:  # C2 and C3 for x-ray
    filename = xray_pattern.format(channel=channel)
    filepath = os.path.join(base_dir, filename)
    if os.path.exists(filepath):
        print(f"Reading x-ray data from {filename}")
        if channel == "2":
            dipole_data, tarr_x = read_trc_data(filepath)
        elif channel == "3":
            xray_data, tarr_x = read_trc_data(filepath)
    else:
        print(f"Warning: Could not find {filename}")

# read data from Bdot scope
base_dir = os.path.join(r"E:\Bdot", date)
for channel in ["1", "2", "3"]:  # C1, C2, and C3 for Bdot
    filename = bdot_pattern.format(channel=channel)
    filepath = os.path.join(base_dir, filename)
    if os.path.exists(filepath):
        print(f"Reading Bdot data from {filename}")
        if channel == "1":
            By_P21, tarr_B = read_trc_data(filepath)
        elif channel == "2":
            Bx_P20, tarr_B = read_trc_data(filepath)
        elif channel == "3":
            By_P20, tarr_B = read_trc_data(filepath)
    else:
        print(f"Warning: Could not find {filename}")

print("\nData loading complete")

Looking for files with p30-z13-x200, shot: 00012
Reading x-ray data from C2--E-ring-p30-z13-x200-xray--00012.trc
Reading data...
Done
Reading x-ray data from C3--E-ring-p30-z13-x200-xray--00012.trc
Reading data...
Done
Reading Bdot data from C1--E-ring-p30-z13-x200-Bdot--00012.trc
Reading data...
Done
Reading Bdot data from C2--E-ring-p30-z13-x200-Bdot--00012.trc
Reading data...
Done
Reading Bdot data from C3--E-ring-p30-z13-x200-Bdot--00012.trc
Reading data...
Done

Data loading complete


In [8]:
# Create Photons detector object
detector = Photons(tarr_x * 1000, -xray_data, threshold_multiplier=5.0, negative_pulses=False)

# Reduce pulses to get pulse properties
detector.reduce_pulses()

# Get pulse arrays
pulse_times, pulse_areas = detector.get_pulse_arrays()

plot_photon_detection(
    time_array=tarr_x,
    signal_data=xray_data,
    pulse_times=pulse_times,
    threshold_level=detector.threshold,
    offset=detector.offset
)


Detected 133379 pulses
Signal baseline: -0.04
Detection threshold: 0.03


In [16]:
bin_centers, counts = plot_counts_per_bin(
    pulse_times=pulse_times,
    pulse_areas=pulse_areas,
    bin_width_ms=0.2)
    # amplitude_min=0.01,
    # amplitude_max=0.3


times, frequencies, stft_result = plot_stft(
    time_array=tarr_B,
    signal=By_P21,
    samples_per_fft=500000,
    overlap_fraction=0.01,
    window='hanning',
    freq_min=150, # MHz
    freq_max=1000 # MHz
)

In [17]:
# Create a new figure with a specific size
fig, ax1 = plt.subplots(figsize=(12, 8))

# Get the time limits from STFT data
t_min = times[0]*1000  # Convert to ms
t_max = times[-1]*1000  # Convert to ms

# Plot STFT spectrogram
im = ax1.imshow(stft_result.T, 
                aspect='auto',
                origin='lower',
                extent=[t_min, t_max, frequencies[0], frequencies[-1]],
                interpolation='nearest',
                cmap='jet')

# Add colorbar for STFT
cbar = plt.colorbar(im, ax=ax1)
cbar.set_label('Magnitude')

# Set labels for frequency axis
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Frequency (MHz)')

# Create second y-axis for counts
ax2 = ax1.twinx()

# Filter counts data to match STFT time range
mask = (bin_centers >= t_min) & (bin_centers <= t_max)
filtered_centers = bin_centers[mask]
filtered_counts = counts[mask]

# Plot filtered counts data
ax2.plot(filtered_centers, filtered_counts, 'w-', linewidth=2, alpha=0.7, label='Photon Counts')

# Set axis limits
ax1.set_xlim(t_min, t_max)

# Set labels for counts axis
ax2.set_ylabel('Counts per Bin', color='w')
ax2.tick_params(axis='y', labelcolor='w')

# Add title
plt.title('STFT Spectrogram with Photon Counts Overlay')

# Add legend for counts plot
ax2.legend(loc='upper right')

plt.show()


In [5]:
# filepath = r"C:\data\LAPD\Bdot_calibration\BDOT-2C-1T-RB1\Raw Cal Data 3Ghz 10-24\bdot-2c-1-bypy-2g.csv"
# data_dict = read_NA_data(filepath)


Dictionary keys: dict_keys(['metadata', 'Freq(Hz)', 'S22', 'S23', 'S24', 'S32', 'S33', 'S34', 'S42', 'S43', 'S44'])
